In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
from jdaviz import SpecViz

specviz = SpecViz()
specviz.app

/Users/rosteen/opt/anaconda3/envs/viz_dev/lib/python3.8/site-packages/glue/external/echo/__init__.py:3: UserWarning: glue.external.echo is deprecated, import from echo directly instead
  warnings.warn('glue.external.echo is deprecated, import from echo directly instead')


Application(components={'g-viewer-tab': '<template>\n  <component :is="stack.container">\n    <g-viewer-tab\n …

In [3]:
from astropy.utils.data import download_file

# This file is originally from https://data.sdss.org/sas/dr14/manga/spectro/redux/v2_1_2/7495/stack/manga-7495-12704-LOGCUBE.fits.gz
# but has been modified to correct some inconsistencies in the way units are parsed
#fn = download_file('https://stsci.box.com/shared/static/28a88k1qfipo4yxc4p4d40v4axtlal8y.fits', cache=True)
#specviz.load_spectrum(fn, "myfile", )

fn = download_file('https://data.sdss.org/sas/dr14/sdss/spectro/redux/26/spectra/0751/spec-0751-52251-0160.fits', cache=True)
specviz.load_spectrum(fn, "Demo SDSS", format="SDSS-III/IV spec")

Spectrum1D (length=3841)
flux:             [ 30.597 1e-17 erg / (Angstrom cm2 s), ..., 51.703 1e-17 erg / (Angstrom cm2 s) ],  mean=51.88 1e-17 erg / (Angstrom cm2 s)
spectral axis:    [ 3799.3 Angstrom, ..., 9198.1 Angstrom ],  mean=6106.1 Angstrom
uncertainty:      [ InverseVariance(0.06440803), ..., InverseVariance(0.18243204) ]


### Adding line lists via code in cells

Line lists are expected to be input as an astropy `QTable`, with at minimum "linename" and "rest" fields. The user can optionally specify a "color" column if they want each line to be a different color. 

The first step is to load the line list into the specviz instance using `specviz.load_line_list(lines)`. A line list as described as above is required, and there is an optional `replace` parameter that, if set to True, replaces any existing lines with those input. The default behavior is to add newly loaded lines to the existing list.

Note that the line list functionality is all actually happening under the hood in the spectrum viewer (and thus could also be used in e.g. Cubeviz); the Specviz methods are convenience wrappers. 

In [ ]:
from astropy.table import QTable
import astropy.units as u

lt = QTable()
lt['linename'] = ['O III','Halpha']
lt['rest'] = [5007, 6563]*u.AA
lt['redshift'] = u.Quantity(0.046)

specviz.load_line_list(lt)
specviz.spectral_lines

If the loaded table has a `listname` column, the lines will show up in the Line List UI under that heading. Otherwise, they will by default show up under the `Custom` heading.

In [ ]:
lt = QTable()
lt['linename'] = ['Test1', 'Test2']
lt['rest'] = [6.8e-7, 4.4e-7]*u.m
lt['redshift'] = u.Quantity(0.046)
lt['listname'] = "Test"

specviz.load_line_list(lt)
specviz.spectral_lines

Adding the lines to the plot is as simple as calling the `plot_spectral_lines` function:

In [ ]:
specviz.plot_spectral_lines()

And one can erase all spectral lines from the plot as below:

In [ ]:
specviz.erase_spectral_lines()

### Preset line lists

We also include with the jdaviz package a set of predefined line lists that can be loaded and plotted by the user. The names of these lists, which can be accessed via `specviz.available_linelists`, can be passed to `specviz.load_line_list` instead of passing in a QTable. The `replace` keyword still works as above.

In [ ]:
specviz.available_linelists

In [ ]:
specviz.load_line_list("Common Nebular", replace=True)

In [ ]:
specviz.spectral_lines

In [ ]:
specviz.plot_spectral_lines()

In [ ]:
specviz.spectral_lines["redshift"] = u.Quantity(0.87)
specviz.spectral_lines["redshift"] 

In [ ]:
specviz.spectral_lines["redshift"] 

In [4]:
d = specviz.app.data_collection[0]

In [6]:
#d.meta["redshift"] = 0.5
d.coords.spectral_axis.redshift

<Quantity 0.22>

In [ ]:
d.get_object()

In [ ]:
from specutils import Spectrum1D
f = Spectrum1D.read(fn)
f

In [ ]:
f.redshift = 1

In [ ]:
f.meta

In [ ]:
specviz.get_spectra()

In [ ]:
from glue_astronomy.spectral_coordinates import SpectralCoordinates
from specutils import SpectralAxis

a = d.coords.spectral_axis
type(a)

In [ ]:
b = SpectralAxis(a, redshift=1.2)
b.redshift

In [ ]:
c = SpectralCoordinates(b)
c.spectral_axis.redshift

In [ ]:
d.coords = c

In [ ]:
d.coords.spectral_axis.redshift

In [9]:
test = specviz.app.get_data_from_viewer("spectrum-viewer")["Demo SDSS"]

In [10]:
test

<Spectrum1D(flux=<Quantity [30.596626, 33.245728, 35.89512 , ..., 53.27969 , 50.236168,
           51.702717] 1e-17 erg / (Angstrom cm2 s)>, spectral_axis=<SpectralAxis 
   (observer to target:
      radial_velocity=58840.4743960778 km / s
      redshift=0.21999999999999997)
  [3799.2686, 3800.1426, 3801.0188, ..., 9193.905 , 9196.0205, 9198.141 ] Angstrom>, uncertainty=InverseVariance([0.06440803, 0.        , 0.06292833, ..., 0.2728394 ,
                 0.2806295 , 0.18243204]))>